<a href="https://colab.research.google.com/github/JoaoBarioni/DeepLearning-STL-10-ResNet/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [39]:

class CustomResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        # Ajustar a última camada fully connected para o número de classes desejado
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.resnet(x)

In [40]:
# Transformações para o conjunto de dados STL-10
transform_stl10 = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [41]:
# Carregamento de dados STL-10
train_dataset_stl10 = datasets.STL10(root='./data', split='train', download=True, transform=transform_stl10)
test_dataset_stl10 = datasets.STL10(root='./data', split='test', download=True, transform=transform_stl10)

train_loader_stl10 = DataLoader(train_dataset_stl10, batch_size=64, shuffle=True, num_workers=2)
test_loader_stl10 = DataLoader(test_dataset_stl10, batch_size=64, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [42]:
# Função para treinar o modelo
def train_resnet(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    return total_loss / len(loader), accuracy


In [43]:

# Função para validar o modelo
def validate_resnet(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    return total_loss / len(loader), accuracy


In [44]:
# Escolher otimizador, scheduler e função de perda como antes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_resnet = CustomResNet(num_classes=10).to(device)
optimizer_resnet = optim.Adam(model_resnet.parameters(), lr=1e-4)
scheduler_resnet = StepLR(optimizer_resnet, step_size=10, gamma=0.8)
criterion_resnet = nn.CrossEntropyLoss()

In [45]:
# Loop de treinamento e validação com cálculo de acurácia
epochs_resnet = 50
train_losses_resnet = []
train_accuracies_resnet = []
test_losses_resnet = []
test_accuracies_resnet = []

In [ ]:
for epoch in range(epochs_resnet):
    train_loss, train_accuracy = train_resnet(model_resnet, train_loader_stl10, optimizer_resnet, criterion_resnet)
    test_loss, test_accuracy = validate_resnet(model_resnet, test_loader_stl10, criterion_resnet)

    scheduler_resnet.step()

    train_losses_resnet.append(train_loss)
    train_accuracies_resnet.append(train_accuracy)
    test_losses_resnet.append(test_loss)
    test_accuracies_resnet.append(test_accuracy)

    print(f"Epoch {epoch + 1}/{epochs_resnet} => "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}% | "
          f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


Epoch 1/50 => Train Loss: 2.1272, Train Accuracy: 39.26% | Test Loss: 1.8785, Test Accuracy: 57.00%
Epoch 2/50 => Train Loss: 1.7840, Train Accuracy: 70.84% | Test Loss: 1.6694, Test Accuracy: 82.75%
Epoch 3/50 => Train Loss: 1.6161, Train Accuracy: 86.44% | Test Loss: 1.6121, Test Accuracy: 85.21%
Epoch 4/50 => Train Loss: 1.5828, Train Accuracy: 89.14% | Test Loss: 1.6136, Test Accuracy: 84.88%
Epoch 5/50 => Train Loss: 1.5646, Train Accuracy: 90.24% | Test Loss: 1.5950, Test Accuracy: 86.72%
Epoch 6/50 => Train Loss: 1.5494, Train Accuracy: 92.20% | Test Loss: 1.5977, Test Accuracy: 86.41%
Epoch 7/50 => Train Loss: 1.5406, Train Accuracy: 92.54% | Test Loss: 1.5898, Test Accuracy: 87.29%
Epoch 8/50 => Train Loss: 1.5315, Train Accuracy: 93.38% | Test Loss: 1.5890, Test Accuracy: 87.19%
Epoch 9/50 => Train Loss: 1.5219, Train Accuracy: 94.28% | Test Loss: 1.5950, Test Accuracy: 86.61%
Epoch 10/50 => Train Loss: 1.5206, Train Accuracy: 94.30% | Test Loss: 1.5896, Test Accuracy: 87.24%